## Getting text data from youtube videos

### Overview

This workbook provides a short illustration of how to download the transcripts for a series of youtube videos, store them in pandas dataframes, and run a few queries to show the kind of analysis you may be interested in doing in your own research. 

### Software

This workbook uses the following modules.

* Pandas
* PandaSQL
* youtube_transcript_api

### Background

In my consulting sessions with researchers, I've been getting more questions lately about how to extract text with time stamps from videos and images. This makes a certain amount of sense, with the amount of public data now posted online. 

Youtube often provides a text transcript (captions) along with time stamps. You can access this transcript without any programming through the youtube website. Here are instructions:

https://ccm.net/faq/40644-how-to-get-the-transcript-of-a-youtube-video

If you have a large number of videos, you may want to avoid a lot of manual downloading and formatting and use a python script. Fortunately, an open source module, "youtube_transcipt_api", provides an easy API for this task.

https://pypi.org/project/youtube-transcript-api/


#### Note - Getting text data from my own videos

If you have videos and don't mind making them public or unlisted, you can use this approach by uploading them to youtube and using the methods here. If you have a very large dataset, you might want to use a cloud storage and API solution. Various platforms provide this - here's a link to the google API.

https://cloud.google.com/video-intelligence/docs/text-detection

This takes a little more programming and configuration, and may result in some cloud computing charges depending depending on the amount of data you want to process, but it is probably more scalable and can offer a more secure environment for videos you want to keep private.  

### Sample Data

This workbook reads the text from a series of youtube videos,
formats them in a python dataframe, and queries them by timestamp and text strings.

For illustration, we'll use a series of lectures from "On Power and Politics in Today's World"

https://www.youtube.com/playlist?list=PLh9mgdi4rNeyViG2ar68jkgEi4y6doNZy"

### Install and import the youtube_transcript_api

You'll need to install the module before you can import it. You only have to do this once on your system (even if you use it in a different notebook or python script), so you may want to comment out or remove this line after running it once. 

In [ ]:
#!pip install youtube_transcript_api

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi
import pandas as pd

In [ ]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = None

### Extract the transcript from one video

We'll extract the transcript for each video using the YouTubeTransciptAPI get_transcript() method. This method takes the video ID as a parameter.

You can get the video ID from the URL on youtube - for example, https://www.youtube.com/watch?v=BDqvzFY72mg has the ID 'BDqvzFY72mg'

In [ ]:
transcript = YouTubeTranscriptApi.get_transcript('BDqvzFY72mg')

The method returns the transcript of the video as a list of lines, each stored as a dictionary. We get 1280 lines from the video above.

In [ ]:
print(type(transcript))
print(len(transcript))

Let's look at the first few lines. Each line contains a dictionary with keys "text", "start", and "duratation". 

In [ ]:
transcript[:3]

You can parse the video using standard techiques for JSON or dictionaries (more info here: https://github.com/geoffswc/Python-JSON-Workshop).

### Transcripts for Multiple Videos in Pandas Format

Fortunately, this is a flat dictionary structure, not deeply nested, so we can convert this to a pandas dataframe easily. In this next section, we'll review code to convert a series of videos and concatenate them into a single data frame.

First, we'll greate a list of IDs for each video. 

In [ ]:
links = [
    'BDqvzFY72mg',
    'f5nbT4xQqwI',
    's48b9B5gd88',
    '4eUS8trd_yI',
    'aKW_Vsk4hzs',
    'q53DF6ySOZg',
    'T3-VlQu3iRM'
]

Extract the transcript for each video using the YouTubeTransciptAPI get_transcript() method. 

For now, we will store the transcript for each video in a list named transcripts. 

In [ ]:
transcripts = []
for v in links:
    try:
        df = pd.DataFrame(YouTubeTranscriptApi.get_transcript(v))
        df['video_id'] = v
        transcripts.append(df)
    except:
        print(v, 'failed to translate')

Note that we have now created a list of pandas dataframes. Let's take a look at a few lines from the first one.

In [ ]:
transcripts[0].head()

Next, we'll combine all the dataframes into a single dataframe.

In [ ]:
df_transcripts = pd.concat(transcripts).reset_index(drop=True)

In [ ]:
#df_transcripts.iloc[1000:1100]

### Run some queries

Now that we have our text in a single dataframe, we can analyze it using a wide range of tools in python. You might be interested in natural language processing, sentiment analysis, text classification, lexical structures, regional differences in language ussed in school board meeings... more than we can get into here (though feel free to get started here with the Library "Document Classification with Scikit-Learn" workshop aat https://courses.ucsf.edu/course/view.php?id=8249)

For now, we'll just query the data in a few ways and leave it there. If you've taken any of my workshops, you'll know I lean toward using SQL, so I'll write a very queries using the pandasql module. 

In [ ]:
# !pip isntall pandasql 
from pandasql import sqldf 
pysqldf = lambda q: sqldf(q, globals())

In [ ]:
# which videos have the most lines of text
pysqldf("SELECT video_id, COUNT(*) FROM df_transcripts GROUP BY video_id")

In [ ]:
# which videos were longest (highest timestamp + duration)
pysqldf("SELECT video_id, MAX(start + duration) FROM df_transcripts GROUP BY video_id")

In [ ]:
# Most mentions of the Cold War
pysqldf("""
SELECT 
    video_id, 
    COUNT(1) 
FROM 
    df_transcripts 
WHERE 
    LOWER(text) LIKE ('%cold war%')
GROUP BY 
    video_id""")

In [ ]:
# what rows matched
pysqldf("""
SELECT 
    *
FROM 
    df_transcripts 
WHERE 
    LOWER(text) LIKE ('%cold war%')
""")

In [ ]:
# when was cold war first mentioned in each video?
pysqldf("""
SELECT 
    video_id, 
    MIN(start) 
FROM 
    df_transcripts 
WHERE 
    LOWER(text) LIKE ('%cold war%')
GROUP BY 
    video_id""")

In [ ]:
#aKW_Vsk4hzs
transcript = pd.DataFrame(YouTubeTranscriptApi.get_transcript('q53DF6ySOZg'))

In [ ]:
transcript

In [ ]:
' '.join(transcript['text'])

In [ ]:
df_transcripts = df_transcripts.replace(r'\n',' ', regex=True)

In [ ]:
df_transcripts.head(20)